# Self-Driving Car Engineer Nanodegree


## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import glob
%matplotlib qt

## 1. Compute camera caliberation

In [2]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

cv2.destroyAllWindows()
# Get Camera Caliberation Matrix and Distortion coefficients
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)


## Helper Functions

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [51]:
import math

def get_undistored_img(img, mtx, dist):
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

def get_sobelx_binary(img, sx_thresh=(60, 160)):
    # get sobel x binary 
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    sx_binary = np.zeros_like(scaled_sobel)
    sx_binary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    return sx_binary
    
def get_r_binary(img, thresh = (200, 255)):    
    R = img[:,:,0]
    binary = np.zeros_like(R)
    binary[(R > thresh[0]) & (R <= thresh[1])] = 1
    return binary
    
def get_s_channel_binary(img, s_thresh=(150, 255)):
    # get S chanel binary
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel > s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    return s_binary
    
def get_combined_binary(img):  
    # get sobel x binary 
    sx_binary = get_sobelx_binary(img)
    
    # get S chanel binary
    s_binary = get_s_channel_binary(img)
    
    combined_binary = np.zeros_like(sx_binary)
    combined_binary[(s_binary == 1) | (sx_binary == 1)] = 1
    
    return combined_binary    


def get_wrapped_binary(img):  
    # get sobel x binary 
    offset = 150
    src = np.float32([[160, img.shape[0]],
           [570, 460],
           [730, 460],
           [img.shape[1] - 110, img.shape[0]]])
    dst = np.float32([[offset, img.shape[0]],
           [offset, offset],
           [img.shape[1] - offset, offset],
           [img.shape[1] - offset, img.shape[0]]])
    
    M = cv2.getPerspectiveTransform(src, dst)   
    img_size = (img.shape[1], img.shape[0])
    warped_binary = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped_binary

def get_unwrapped_img(img):  
    # get sobel x binary 
    offset = 150
    src = np.float32([[160, img.shape[0]],
           [570, 460],
           [730, 460],
           [img.shape[1] - 110, img.shape[0]]])
    dst = np.float32([[offset, img.shape[0]],
           [offset, offset],
           [img.shape[1] - offset, offset],
           [img.shape[1] - offset, img.shape[0]]])
    
    M = cv2.getPerspectiveTransform(dst, src)   
    img_size = (img.shape[1], img.shape[0])
    warped_binary = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped_binary
    
# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)


def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        ### TO-DO: Find the four below boundaries of the window ###
        win_xleft_low = leftx_current - margin  # Update this
        win_xleft_high = leftx_current + margin  # Update this
        win_xright_low = rightx_current - margin  # Update this
        win_xright_high = rightx_current + margin  # Update this
        
        # Draw the windows on the visualization image
        # cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        # (win_xleft_high,win_y_high),(0,255,0), 2) 
        # cv2.rectangle(out_img,(win_xright_low,win_y_low),
        # (win_xright_high,win_y_high),(0,255,0), 2) 
        
        ### Identify the nonzero pixels in x and y within the window ###
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        ### TO-DO: If you found > minpix pixels, recenter next window ###
        ### (`right` or `leftx_current`) on their mean position ###
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty


def visualize_output(left_fitx, right_fitx, ploty, out_img):
     
    margin = 0
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    lane_line_pts = np.hstack((left_line_window2, right_line_window1))
    window_img = np.zeros_like(out_img)
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([lane_line_pts]), (0,255, 0))
    #result = out_img
    result = cv2.addWeighted(out_img, 0.8, window_img, 1.0, 0)
    return result 

def get_poly_fit(img_shape, leftx, lefty, rightx, righty):
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    return left_fit, right_fit, ploty  

def fit_poly(left_fit, right_fit, ploty):
    
    #Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fitx, right_fitx, ploty

def search_around_poly(binary_warped, left_fit, right_fit):
    # HYPERPARAMETER
    # Choose the width of the margin around the previous polynomial to search
    # The quiz grader expects 100 here, but feel free to tune on your own!
    margin = 100

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    ### Set the area of search based on activated x-values ###
    ### within the +/- margin of our polynomial function ###
    ### Hint: consider the window areas for the similarly named variables ###
    ### in the previous quiz, but change the windows to our new search area ###
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                    left_fit[1]*nonzeroy + left_fit[2] + margin)))
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                    right_fit[1]*nonzeroy + right_fit[2] + margin)))
    
    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty


def measure_curvature_real(leftx, rightx, lefty, righty):
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    
    left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    
    # Implement the calculation of R_curve (radius of curvature) #####
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    return left_curverad, right_curverad 

def get_vehicle_position(centerx, leftx, rightx):
    xm_per_pix = 3.7/700
    midLane = (np.mean(leftx) + np.mean(rightx))/2
    position = midLane - centerx 
    return position * xm_per_pix


def put_text(image, text1, text2):
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org1 = (50, 50)
    org2 = (50, 100)
    fontScale = 1
    color = (255, 0, 0) 
    thickness = 2
    image = cv2.putText(image, text1, org1, font,  
                       fontScale, color, thickness, cv2.LINE_AA)
    image = cv2.putText(image, text2, org2, font,  
                       fontScale, color, thickness, cv2.LINE_AA)
    return image


# Define a class lane boundary detection
class LaneBoundary():
    def __init__(self):
        self.find_lanes = True
        # was error Counter
        self.error_counter = 0  
        #radius of curvature of the line in some units
        self.last_valid_left_curvature = None
        self.last_valid_right_curvature = None
        self.last_valid_leftx = None
        self.last_valid_lefty = None
        self.last_valid_rightx = None
        self.last_valid_righty = None
        self.valid_left_fit = None
        self.valid_right_fit = None
            
    def set_valid_poly_fit(self, left_fit, right_fit):
        self.valid_left_fit = left_fit
        self.valid_right_fit = right_fit
            
    def detect_lane_pixels(self, wraped_binary):
        if(self.find_lanes):
            self.find_lanes = False;
            
            leftx, lefty, rightx, righty = find_lane_pixels(wraped_binary)
            self.last_valid_leftx = leftx
            self.last_valid_lefty = lefty
            self.last_valid_rightx = rightx
            self.last_valid_righty = righty
        else:
            leftx, lefty, rightx, righty = search_around_poly(wraped_binary, self.valid_left_fit, self.valid_right_fit)
            self.sanity_check(leftx, lefty, rightx, righty)    
        return self.last_valid_leftx, self.last_valid_lefty, self.last_valid_rightx, self.last_valid_righty
    
    def sanity_check(self, leftx, lefty, rightx, righty):
        left_curverad, right_curverad = measure_curvature_real(leftx, rightx, lefty, righty)        
        threshold = 50
        diffleft = np.absolute(left_curverad - self.last_valid_left_curvature)
        diffright = np.absolute(right_curverad - self.last_valid_right_curvature)
        if((diffleft > threshold) or (diffright > threshold)):
            self.error_counter = self.error_counter + 1
        else:
            self.last_valid_leftx = leftx
            self.last_valid_lefty = lefty
            self.last_valid_rightx = rightx
            self.last_valid_righty = righty
            self.error_counter = 0

        if(self.error_counter >= 3):
            self.error_counter = 0
            self.find_lanes = True
         
    def set_lane_curvature(self, left_curverad, right_curverad):
        self.last_valid_left_curvature = left_curverad
        self.last_valid_right_curvature = right_curverad


## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [10]:
import os
testImages = os.listdir("test_images/")

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [65]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

for imageName in testImages:
    imagePath = 'test_images/' + imageName
    
    image = mpimg.imread(imagePath)
    
    undist = get_undistored_img(image, mtx, dist)
    
    binary = get_combined_binary(undist)
    
    wraped_binary = get_wrapped_binary(binary)
    
    leftx, lefty, rightx, righty = find_lane_pixels(wraped_binary)
    
    left_fit, right_fit, ploty = get_poly_fit(wraped_binary.shape, leftx, lefty, rightx, righty)
   
    left_fitx, right_fitx, ploty = fit_poly(left_fit, right_fit, ploty)
    
    left_curverad, right_curverad = measure_curvature_real(leftx, rightx, lefty, righty)
    position = get_vehicle_position(binary.shape[1]/2, leftx, rightx)
    
    mean_curvature = (left_curverad + right_curverad)/2
    text1 = 'Radius of curvature ' + str(mean_curvature)
    text2 = 'Position of vehicle ' + str(position) 
    
    out_img = np.dstack((wraped_binary, wraped_binary, wraped_binary))
    
    result = visualize_output(left_fitx, right_fitx, ploty, out_img)
    
    unwrapped = get_unwrapped_img(result)
    
    annoted_output = weighted_img(unwrapped, image)
    
    output = put_text(annoted_output, text1, text2)
    
    filename = 'output_images/output_' + imageName 
    cv2.imwrite(filename, output) 
    

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [47]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [53]:
laneBoundary = LaneBoundary()
def process_image(image):
    
    undist = get_undistored_img(image, mtx, dist)
    
    binary = get_combined_binary(undist)
    
    wraped_binary = get_wrapped_binary(binary)
    
    leftx, lefty, rightx, righty = laneBoundary.detect_lane_pixels(wraped_binary)
    
    left_fit, right_fit, ploty = get_poly_fit(wraped_binary.shape, leftx, lefty, rightx, righty)
    laneBoundary.set_valid_poly_fit(left_fit, right_fit)

    left_fitx, right_fitx, ploty = fit_poly(left_fit, right_fit, ploty)
    
    left_curverad, right_curverad = measure_curvature_real(leftx, rightx, lefty, righty)
    laneBoundary.set_lane_curvature(left_curverad, right_curverad)
    position = get_vehicle_position(binary.shape[1]/2, leftx, rightx) 
    
    mean_curvature = (left_curverad + right_curverad)/2
    text1 = 'Radius of curvature ' + str(mean_curvature)
    text2 = 'Position of vehicle ' + str(position) 
    
    out_img = np.dstack((wraped_binary, wraped_binary, wraped_binary))
    
    result = visualize_output(left_fitx, right_fitx, ploty, out_img)
    
    unwrapped = get_unwrapped_img(result)
    
    annoted_output = weighted_img(unwrapped, image)
    
    output = put_text(annoted_output, text1, text2)
    
    return output

Let's try the one with the solid white lane on the right first ...

In [54]:
white_output = 'test_output_videos/output_project_video.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)




                                                                   

                                                             



                                                       




                                                   
                                                                


t:   0%|          | 1/1260 [43:49<919:44:35, 2629.93s/it, now=None]

t:   0%|          | 1/1260 [33:49<709:42:53, 2029.37s/it, now=None]



t:   0%|          | 1/1260 [22:48<478:44:57, 1368.94s/it, now=None]




t:   0%|          | 1/1260 [41:27<870:04:29, 2487.90s/it, now=None]


                                                                   

                                                             



                                                       




                                                    
                                                                


t:   0%|          | 1/1260 [43:49<919:45:14, 2629.96s/it, now=None]

t:   0%|          |

Moviepy - Building video test_output_videos/output_project_video.mp4.
Moviepy - Writing video test_output_videos/output_project_video.mp4









t:   0%|          | 3/1260 [00:00<01:33, 13.44it/s, now=None]





t:   0%|          | 4/1260 [00:00<01:50, 11.36it/s, now=None]





t:   0%|          | 6/1260 [00:00<01:45, 11.90it/s, now=None]





t:   1%|          | 8/1260 [00:00<01:43, 12.08it/s, now=None]





t:   1%|          | 10/1260 [00:00<01:44, 11.99it/s, now=None]





t:   1%|          | 12/1260 [00:00<01:44, 11.96it/s, now=None]





t:   1%|          | 14/1260 [00:01<01:42, 12.11it/s, now=None]





t:   1%|▏         | 16/1260 [00:01<01:43, 11.99it/s, now=None]





t:   1%|▏         | 18/1260 [00:01<01:41, 12.25it/s, now=None]





t:   2%|▏         | 20/1260 [00:01<01:40, 12.34it/s, now=None]





t:   2%|▏         | 22/1260 [00:01<01:42, 12.06it/s, now=None]





t:   2%|▏         | 24/1260 [00:01<01:39, 12.43it/s, now=None]





t:   2%|▏         | 26/1260 [00:02<01:37, 12.61it/s, now=None]





t:   2%|▏         | 28/1260 [00:02<01:36, 12.72it/s, now=None]





t:   2%|▏         | 30/1260 [00:02<01:35, 12.9

t:  13%|█▎        | 167/1260 [00:15<02:16,  7.98it/s, now=None]





t:  13%|█▎        | 168/1260 [00:15<02:24,  7.55it/s, now=None]





t:  13%|█▎        | 169/1260 [00:15<02:19,  7.82it/s, now=None]





t:  13%|█▎        | 170/1260 [00:15<02:12,  8.24it/s, now=None]





t:  14%|█▎        | 171/1260 [00:15<02:11,  8.25it/s, now=None]





t:  14%|█▎        | 172/1260 [00:15<02:17,  7.88it/s, now=None]





t:  14%|█▎        | 173/1260 [00:15<02:16,  7.94it/s, now=None]





t:  14%|█▍        | 175/1260 [00:16<02:08,  8.45it/s, now=None]





t:  14%|█▍        | 176/1260 [00:16<02:09,  8.36it/s, now=None]





t:  14%|█▍        | 177/1260 [00:16<02:08,  8.40it/s, now=None]





t:  14%|█▍        | 178/1260 [00:16<02:03,  8.74it/s, now=None]





t:  14%|█▍        | 179/1260 [00:16<02:01,  8.91it/s, now=None]





t:  14%|█▍        | 180/1260 [00:16<02:03,  8.73it/s, now=None]





t:  14%|█▍        | 181/1260 [00:16<02:00,  8.97it/s, now=None]





t:  14%|█▍        | 182/1260 [00:1

t:  21%|██        | 266/1260 [00:26<01:53,  8.77it/s, now=None]





t:  21%|██        | 267/1260 [00:26<01:51,  8.89it/s, now=None]





t:  21%|██▏       | 268/1260 [00:27<01:57,  8.44it/s, now=None]





t:  21%|██▏       | 269/1260 [00:27<01:56,  8.53it/s, now=None]





t:  21%|██▏       | 270/1260 [00:27<01:52,  8.81it/s, now=None]





t:  22%|██▏       | 271/1260 [00:27<01:49,  9.00it/s, now=None]





t:  22%|██▏       | 272/1260 [00:27<01:53,  8.71it/s, now=None]





t:  22%|██▏       | 273/1260 [00:27<01:56,  8.51it/s, now=None]





t:  22%|██▏       | 274/1260 [00:27<01:52,  8.80it/s, now=None]





t:  22%|██▏       | 275/1260 [00:27<01:50,  8.88it/s, now=None]





t:  22%|██▏       | 276/1260 [00:28<02:01,  8.08it/s, now=None]





t:  22%|██▏       | 277/1260 [00:28<02:01,  8.07it/s, now=None]





t:  22%|██▏       | 279/1260 [00:28<01:55,  8.53it/s, now=None]





t:  22%|██▏       | 280/1260 [00:28<01:56,  8.41it/s, now=None]





t:  22%|██▏       | 281/1260 [00:2

...

t:  78%|███████▊  | 987/1260 [01:50<00:31,  8.74it/s, now=None]





t:  78%|███████▊  | 988/1260 [01:50<00:31,  8.76it/s, now=None]





t:  78%|███████▊  | 989/1260 [01:50<00:30,  8.91it/s, now=None]





t:  79%|███████▊  | 990/1260 [01:50<00:30,  8.97it/s, now=None]





t:  79%|███████▊  | 991/1260 [01:51<00:32,  8.36it/s, now=None]





t:  79%|███████▊  | 992/1260 [01:51<00:30,  8.71it/s, now=None]





t:  79%|███████▉  | 993/1260 [01:51<00:30,  8.68it/s, now=None]





t:  79%|███████▉  | 994/1260 [01:51<00:29,  8.94it/s, now=None]





t:  79%|███████▉  | 995/1260 [01:51<00:31,  8.31it/s, now=None]





t:  79%|███████▉  | 996/1260 [01:51<00:31,  8.49it/s, now=None]





t:  79%|███████▉  | 998/1260 [01:51<00:29,  8.88it/s, now=None]





t:  79%|███████▉  | 999/1260 [01:52<00:33,  7.91it/s, now=None]





t:  79%|███████▉  | 1000/1260 [01:52<00:32,  8.03it/s, now=None]





t:  80%|███████▉  | 1002/1260 [01:52<00:30,  8.58it/s, now=None]





t:  80%|███████▉  | 1003/1260 [0

t:  86%|████████▌ | 1082/1260 [02:02<00:25,  7.04it/s, now=None]





t:  86%|████████▌ | 1083/1260 [02:02<00:24,  7.31it/s, now=None]





t:  86%|████████▌ | 1084/1260 [02:02<00:23,  7.43it/s, now=None]





t:  86%|████████▌ | 1085/1260 [02:03<00:22,  7.66it/s, now=None]





t:  86%|████████▌ | 1086/1260 [02:03<00:23,  7.45it/s, now=None]





t:  86%|████████▋ | 1087/1260 [02:03<00:23,  7.49it/s, now=None]





t:  86%|████████▋ | 1088/1260 [02:03<00:23,  7.28it/s, now=None]





t:  86%|████████▋ | 1089/1260 [02:03<00:23,  7.27it/s, now=None]





t:  87%|████████▋ | 1090/1260 [02:03<00:23,  7.23it/s, now=None]





t:  87%|████████▋ | 1091/1260 [02:03<00:23,  7.33it/s, now=None]





t:  87%|████████▋ | 1092/1260 [02:03<00:22,  7.34it/s, now=None]





t:  87%|████████▋ | 1093/1260 [02:04<00:22,  7.54it/s, now=None]





t:  87%|████████▋ | 1094/1260 [02:04<00:20,  7.93it/s, now=None]





t:  87%|████████▋ | 1095/1260 [02:04<00:19,  8.27it/s, now=None]





t:  87%|████████▋ | 

t:  93%|█████████▎| 1178/1260 [02:13<00:09,  8.96it/s, now=None]





t:  94%|█████████▎| 1179/1260 [02:14<00:09,  8.77it/s, now=None]





t:  94%|█████████▎| 1180/1260 [02:14<00:08,  8.89it/s, now=None]





t:  94%|█████████▎| 1181/1260 [02:14<00:09,  8.66it/s, now=None]





t:  94%|█████████▍| 1182/1260 [02:14<00:09,  8.42it/s, now=None]





t:  94%|█████████▍| 1183/1260 [02:14<00:08,  8.60it/s, now=None]





t:  94%|█████████▍| 1184/1260 [02:14<00:08,  8.53it/s, now=None]





t:  94%|█████████▍| 1186/1260 [02:14<00:08,  8.47it/s, now=None]





t:  94%|█████████▍| 1187/1260 [02:15<00:08,  8.43it/s, now=None]





t:  94%|█████████▍| 1189/1260 [02:15<00:08,  8.82it/s, now=None]





t:  94%|█████████▍| 1190/1260 [02:15<00:08,  8.39it/s, now=None]





t:  95%|█████████▍| 1191/1260 [02:15<00:08,  8.39it/s, now=None]





t:  95%|█████████▍| 1192/1260 [02:15<00:07,  8.66it/s, now=None]





t:  95%|█████████▍| 1193/1260 [02:15<00:07,  8.75it/s, now=None]





t:  95%|█████████▍| 

                                                                


t:   0%|          | 1/1260 [46:14<970:16:01, 2774.39s/it, now=None]

t:   0%|          | 1/1260 [36:13<760:14:18, 2173.84s/it, now=None]



t:   0%|          | 1/1260 [25:13<529:16:19, 1513.41s/it, now=None]




t:   0%|          | 1/1260 [43:52<920:35:52, 2632.37s/it, now=None]

Moviepy - Done !
Moviepy - video ready test_output_videos/output_project_video.mp4
CPU times: user 18min 36s, sys: 8.69 s, total: 18min 45s
Wall time: 2min 24s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [55]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

Now for the one with the solid yellow lane on the left. This one's more tricky!

## Writeup and Submission

Please find the writeup in the same folder. 
